# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Publish-Subscribe (Pub-Sub)

**Subtopic** - Providing a method to remove all subscribers (reset the system) and using it in a test scenario to show control over subscription management

**Use Case** - Implement a Python-based publish-subscribe system where a publisher object broadcasts messages to subscriber objects. Develop a method within the publisher class to remove all subscribers, effectively resetting the system. This method should clear the internal list of subscribers within the publisher object. To demonstrate control over subscription management, create a scenario where multiple subscriber objects are added to the publisher, messages are broadcasted, and then all subscribers are removed using the developed method. After the removal of subscribers, verify that no further messages are received by the previously subscribed objects.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

You're building a high-performance event dispatch system where multiple listener functions (called subscribers) can register themselves to receive messages broadcasted by a central object called a Publisher. Each subscriber can specify how long it should stay active, what kind of messages it wants to receive, and whether it should receive only once. All operations need to be thread-safe so that the system works correctly even if broadcasts or subscriptions happen from different threads at the same time.

Subscribers are registered by passing in a callable function. When a message is broadcasted, every registered and eligible subscriber must receive it in a specific order based on their priority. After processing the message, some subscribers may automatically remove themselves if they were configured to fire only once or expire after a few broadcasts.

The system must also support safely removing subscribers at any time, either manually or via a reset that clears the entire registry.

### Input Format

All input happens through method calls on a `Publisher` object. Each subscriber is a Python callable that accepts a single argument (the message).

There is no stdin or direct text input.

### Output Format

When calling `.broadcast(message)`, it returns a list of return values from each subscriber (in the order they were called). If any subscriber raises an exception, all other subscribers must still be called, and then the broadcast raises a `SubscriberError` containing the collected exceptions.

The `.audit_log` property returns a list of timestamped mutation logs as strings.



## Examples

```python
pub = Publisher()

def a(msg): return f"A:{msg}"
token = pub.subscribe(a, priority=5)

with pub.temporary(lambda m: f"B:{m}", once=True):
    pub.broadcast("hello")   # calls A and B
    pub.broadcast("world")   # calls A only

pub.unsubscribe(token)
pub.broadcast("bye")         # nothing is called

pub.reset()
pub.broadcast("final")       # nothing is called
```



# Requirements

### Explicit and Implicit Behavior

* A subscriber is a callable that takes one message and returns a result.
* Subscribers can be configured with:

  * `priority` (lower numbers go first),
  * `once=True` (removes itself after one broadcast),
  * `ttl=n` (removes itself after `n` broadcasts),
  * `filter` function (only receive messages passing the predicate).
* Callbacks are delivered in strict priority order, stable for ties.
* A weak reference is used for each subscriber. If the function is garbage collected, it is skipped and removed silently.
* The `.reset()` method clears all subscribers immediately.
* `.unsubscribe(token)` is safe to call at any time, and it silently ignores unknown tokens.
* The `.temporary(fn)` method registers a subscriber for the duration of a `with` block and auto-unsubscribes after exit.
* All internal state is protected by `threading.RLock` to allow safe re-entrant and concurrent use.
* An `OverflowError` is raised if subscribing exceeds `max_size` (if set).
* An `audit_log` is maintained showing every subscribe, unsubscribe, and reset event with precise timing.

### Solution Expectations

* Must be a single class: `Publisher`
* All internal subscribers must be managed using a priority queue with stable insertion ordering.
* Efficient snapshotting must avoid repeated sorting if no change has occurred.
* Dead weakrefs and expired subscribers (due to TTL or once) are cleaned safely.
* No memory leaks or leftover references after reset.

### Function Signatures

```python
class Publisher:
    def __init__(self, *, max_size: Optional[int] = None) -> None: ...
    def subscribe(self, fn: Callable[[Any], Any], *, priority: int = 100, once: bool = False, ttl: Optional[int] = None, filter: Optional[Callable[[Any], bool]] = None) -> int: ...
    def unsubscribe(self, token: int) -> None: ...
    def reset(self) -> None: ...
    def broadcast(self, msg: Any) -> List[Any]: ...
    def temporary(self, fn: Callable[[Any], Any], **kwargs): ...
    audit_log: List[str]
```

### Edge Case Behavior

* Subscribing a non-callable raises `TypeError`.
* Broadcasting with no subscribers must return an empty list.
* If a subscriber raises, all others are still called and a `SubscriberError` is raised.
* Multiple exceptions are aggregated and re-thrown.
* Dead weakrefs are skipped and lazily removed.
* Snapshots are versioned for efficient broadcast re-use.

### Constraints

* Must use only the Python standard library (no third-party dependencies).
* Must run on CPython 3.9+.
* Must not mutate internal state while holding locks during callback execution.
* Must not leak memory when callbacks expire or are deleted.
* Performance must scale to 10,000+ subscribers under multithreaded load.

### Validation Notes

You can assume all inputs satisfy the format no need to validate whether callables or parameters are malformed. If something invalid is passed (e.g., a non-callable), it’s acceptable to raise the standard Python exception (`TypeError`, `OverflowError`, etc.) directly.



In [ ]:
# code

from __future__ import annotations

import heapq
import itertools
import threading
import time
import weakref
import types
from contextlib import contextmanager
from dataclasses import dataclass
from types import TracebackType
from typing import Any, Callable, Dict, List, Optional, Tuple


class SubscriberError(RuntimeError):
    """Raised after broadcast if one or more subscribers raised."""
    def __init__(self, *exc_info: Tuple[BaseException, TracebackType]) -> None:
        super().__init__(f"{len(exc_info)} subscriber(s) raised")
        self.exc_info = exc_info


@dataclass(slots=True)
class _Entry:
    priority: int
    index: int
    token: int
    once: bool = False
    ttl: Optional[int] = None
    filter: Optional[Callable[[Any], bool]] = None
    fn_ref: Callable[[], Optional[Callable[[Any], Any]]] = lambda: None

    def __lt__(self, other: _Entry, /) -> bool:
        return (self.priority, self.index) < (other.priority, other.index)


class Publisher:
    _token_src = itertools.count(1)
    _index_src = itertools.count()

    def __init__(self, *, max_size: Optional[int] = None) -> None:
        self._lock = threading.RLock()
        self._subs: Dict[int, _Entry] = {}
        self._heap: List[_Entry] = []
        self._version = 0
        self._snapshot_cache: Optional[Tuple[int, List[_Entry]]] = None
        self._max_size = max_size
        self.audit_log: List[str] = []

    def _log(self, event: str) -> None:
        ts = time.perf_counter_ns()
        self.audit_log.append(f"{ts}:{event}")

    def _purge_heap(self) -> None:
        alive = [
            e for e in self._heap
            if e.token in self._subs and e.fn_ref() is not None
        ]
        if len(alive) != len(self._heap):
            self._heap[:] = alive
            heapq.heapify(self._heap)
            self._snapshot_cache = None
            self._version += 1

    def _snapshot(self) -> List[_Entry]:
        with self._lock:
            self._purge_heap()
            if self._snapshot_cache and self._snapshot_cache[0] == self._version:
                return list(self._snapshot_cache[1])
            ordered = heapq.nsmallest(len(self._heap), self._heap)
            self._snapshot_cache = (self._version, ordered)
            return ordered.copy()

    def _remove(self, token: int, *, log_event: bool = True) -> None:
        entry = self._subs.pop(token, None)
        if not entry:
            return
        entry.fn_ref = lambda: None
        self._version += 1
        if log_event:
            self._log(f"unsubscribe:{token}")

    def subscribe(
        self,
        fn: Callable[[Any], Any],
        *,
        priority: int = 100,
        once: bool = False,
        ttl: Optional[int] = None,
        filter: Optional[Callable[[Any], bool]] = None,
    ) -> int:
        if not callable(fn):
            raise TypeError("Subscriber must be callable")
        with self._lock:
            if self._max_size is not None and len(self._subs) >= self._max_size:
                raise OverflowError("subscriber capacity reached")
            token = next(self._token_src)
            idx = next(self._index_src)

            # methods held weakly, plain functions strongly, callable objects weakly
            if isinstance(fn, types.MethodType):
                fn_ref = weakref.WeakMethod(fn)
            elif isinstance(fn, types.FunctionType):
                fn_ref = (lambda fn=fn: fn)
            else:
                fn_ref = weakref.ref(fn)

            entry = _Entry(priority, idx, token, once, ttl, filter, fn_ref)
            self._subs[token] = entry
            heapq.heappush(self._heap, entry)
            self._version += 1
            self._log(f"subscribe:{token}")
            return token

    def unsubscribe(self, token: int) -> None:
        with self._lock:
            self._remove(token)

    def reset(self) -> None:
        with self._lock:
            if not self._subs:
                return
            self._subs.clear()
            self._heap.clear()
            self._snapshot_cache = None
            self._version += 1
            self._log("reset")

    def broadcast(self, msg: Any) -> List[Any]:
        entries = self._snapshot()
        results: List[Any] = []
        errors: List[Tuple[BaseException, TracebackType]] = []

        for entry in entries:
            fn = entry.fn_ref()
            if fn is None:
                continue
            if entry.filter and not entry.filter(msg):
                continue
            try:
                results.append(fn(msg))
            except BaseException as exc:
                errors.append((exc, exc.__traceback__))
            finally:
                with self._lock:
                    if entry.once:
                        self._remove(entry.token, log_event=True)
                    elif entry.ttl is not None:
                        entry.ttl -= 1
                        if entry.ttl <= 0:
                            self._remove(entry.token, log_event=True)

        if errors:
            raise SubscriberError(*errors)
        return results

    @contextmanager
    def temporary(self, fn: Callable[[Any], Any], **kwargs):
        token = self.subscribe(fn, **kwargs)
        try:
            yield token
        finally:
            self.unsubscribe(token)


In [ ]:
# tests
import unittest
import threading
import gc
import weakref
from main import Publisher, SubscriberError


class TestPublisherCore(unittest.TestCase):
    def test_subscribe_non_callable(self):
        pub = Publisher()
        with self.assertRaises(TypeError):
            pub.subscribe(123)

    def test_broadcast_no_subscribers(self):
        pub = Publisher()
        self.assertEqual(pub.broadcast("x"), [])

    def test_priority_ordering_and_ties(self):
        pub = Publisher()
        results = []
        pub.subscribe(lambda m: results.append(("low", m)), priority=50)
        pub.subscribe(lambda m: results.append(("mid1", m)), priority=100)
        pub.subscribe(lambda m: results.append(("mid2", m)), priority=100)
        pub.subscribe(lambda m: results.append(("high", m)), priority=150)
        out = pub.broadcast("msg")
        self.assertEqual([r[0] for r in results], ["low", "mid1", "mid2", "high"])
        self.assertEqual(out, [None, None, None, None])

    def test_once_semantics_and_log(self):
        pub = Publisher()
        calls = []
        token = pub.subscribe(lambda m: calls.append(m), once=True)
        pub.broadcast(1)
        pub.broadcast(2)
        # only first call
        self.assertEqual(calls, [1])
        # automatic unsubscribe must be logged
        ops = [e.split(":", 1)[1] for e in pub.audit_log]
        self.assertIn(f"unsubscribe:{token}", ops)

    def test_ttl_semantics_and_log(self):
        pub = Publisher()
        calls = []
        token = pub.subscribe(lambda m: calls.append(m), ttl=2)
        pub.broadcast("a")
        pub.broadcast("b")
        pub.broadcast("c")
        # only first two calls
        self.assertEqual(calls, ["a", "b"])
        # automatic unsubscribe must be logged
        ops = [e.split(":", 1)[1] for e in pub.audit_log]
        self.assertIn(f"unsubscribe:{token}", ops)

    def test_filter_predicate(self):
        pub = Publisher()
        calls = []
        pub.subscribe(lambda m: calls.append(m), filter=lambda x: x % 2 == 0)
        pub.broadcast(1)
        pub.broadcast(2)
        pub.broadcast(3)
        pub.broadcast(4)
        self.assertEqual(calls, [2, 4])

    def test_return_value_propagation(self):
        pub = Publisher()
        pub.subscribe(lambda m: m * 2, priority=20)
        pub.subscribe(lambda m: str(m), priority=10)
        # lower priority value first
        out = pub.broadcast(3)
        self.assertEqual(out, ["3", 6])

    def test_unsubscribe_unknown_no_error(self):
        pub = Publisher()
        pub.unsubscribe(999)  # no-op
        self.assertEqual(pub.audit_log, [])

    def test_reset_idempotent_and_log(self):
        pub = Publisher()
        pub.subscribe(lambda m: None)
        pub.reset()
        ops = [e.split(":", 1)[1] for e in pub.audit_log]
        self.assertIn("reset", ops)
        before = len(pub.audit_log)
        pub.reset()
        self.assertEqual(len(pub.audit_log), before)

    def test_overflow_max_size(self):
        pub = Publisher(max_size=1)
        pub.subscribe(lambda m: None)
        with self.assertRaises(OverflowError):
            pub.subscribe(lambda m: None)

    def test_temporary_context_manager(self):
        pub = Publisher()
        calls = []
        def fn(m): calls.append(m)
        with pub.temporary(fn, once=True):
            pub.broadcast("x")
        pub.broadcast("y")
        self.assertEqual(calls, ["x"])

    def test_weakref_cleanup_for_callable_object(self):
        class A:
            def __call__(self, m): return True
        obj = A()
        pub = Publisher()
        token = pub.subscribe(obj)
        ref = weakref.ref(obj)
        del obj
        gc.collect()
        out = pub.broadcast("z")
        self.assertEqual(out, [])
        self.assertIsNone(ref())

    def test_weakref_cleanup_for_bound_method(self):
        class B:
            def method(self, m): return m
        b = B()
        pub = Publisher()
        token = pub.subscribe(b.method)
        ref = weakref.ref(b)
        del b
        gc.collect()
        out = pub.broadcast("hello")
        self.assertEqual(out, [])
        self.assertIsNone(ref())

    def test_exception_aggregation(self):
        pub = Publisher()
        pub.subscribe(lambda m: (_ for _ in ()).throw(ValueError("err1")))
        called = []
        pub.subscribe(lambda m: called.append("ok"))
        with self.assertRaises(SubscriberError) as cm:
            pub.broadcast("x")
        # second subscriber was still called
        self.assertEqual(called, ["ok"])
        exc = cm.exception
        self.assertEqual(len(exc.exc_info), 1)

    def test_audit_log_sequence(self):
        pub = Publisher()
        t1 = pub.subscribe(lambda m: None)
        t2 = pub.subscribe(lambda m: None)
        pub.unsubscribe(t1)
        pub.reset()
        ops = [e.split(":", 1)[1] for e in pub.audit_log]
        expected = [f"subscribe:{t1}", f"subscribe:{t2}",
                    f"unsubscribe:{t1}", "reset"]
        self.assertEqual(ops, expected)


class TestPublisherConcurrency(unittest.TestCase):
    def test_concurrent_broadcast_and_reset(self):
        pub = Publisher()
        pub.subscribe(lambda m: None)
        errors = []
        stop = threading.Event()

        def spam_broadcast():
            while not stop.is_set():
                try:
                    pub.broadcast("h")
                except Exception as e:
                    errors.append(e)

        t = threading.Thread(target=spam_broadcast)
        t.start()
        threading.Timer(0.05, pub.reset).start()
        threading.Timer(0.1, stop.set).start()
        t.join(1)
        self.assertFalse(errors)
        self.assertEqual(pub.broadcast("x"), [])


if __name__ == "__main__":
    unittest.main(argv=[""], exit=False)


..............
----------------------------------------------------------------------
Ran 14 tests in 0.256s

OK


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/9ed9f24a-b1c1-4cd8-8856-1e227dc96085?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F67966%2Fview

#### Model code:

```python
from __future__ import annotations

import heapq
import itertools
import threading
import time
import weakref
import types
from contextlib import contextmanager
from dataclasses import dataclass
from types import TracebackType
from typing import Any, Callable, Dict, List, Optional, Tuple

__all__ = ["Publisher", "SubscriberError"]

class SubscriberError(RuntimeError):
    """Raised after broadcast if one or more subscribers threw an exception."""
    def __init__(self, *exc_info: Tuple[BaseException, TracebackType]):
        super().__init__(f"{len(exc_info)} subscriber(s) raised")
        self.exc_info = exc_info

@dataclass(slots=True)
class _Entry:
    priority: int
    index: int
    token: int
    once: bool = False
    ttl: Optional[int] = None
    filter: Optional[Callable[[Any], bool]] = None  # <Issue>: name shadows built-in filter
    fn_ref: Callable[[], Optional[Callable[[Any], Any]]] = lambda: None

    def __lt__(self, other: _Entry, /) -> bool:
        return (self.priority, self.index) < (other.priority, other.index)

class Publisher:
    _token_src = itertools.count(1)
    _index_src = itertools.count()

    def __init__(self, *, max_size: Optional[int] = None) -> None:
        self._lock = threading.RLock()
        self._subs: Dict[int, _Entry] = {}
        self._heap: List[_Entry] = []
        self._version = 0
        self._snapshot_cache: Optional[Tuple[int, List[_Entry]]] = None
        self._max_size = max_size
        self.audit_log: List[str] = []

    def _log(self, event: str) -> None:
        self.audit_log.append(f"{time.perf_counter_ns()}:{event}")

    def _snapshot(self) -> List[_Entry]:
        if self._snapshot_cache and self._snapshot_cache[0] == self._version:
            return list(self._snapshot_cache[1])
        ordered = heapq.nsmallest(len(self._heap), self._heap)
        self._snapshot_cache = (self._version, ordered)
        return ordered.copy()

    def _remove(self, token: int, *, log_event: bool = True) -> None:
        entry = self._subs.pop(token, None)
        if entry:
            entry.fn_ref = lambda: None  # <Issue>: leaves stale entries in heap
            self._version += 1
            if log_event:
                self._log(f"unsubscribe:{token}")

    def subscribe(
        self,
        fn: Callable[[Any], Any],
        *,
        priority: int = 100,
        once: bool = False,
        ttl: Optional[int] = None,
        filter: Optional[Callable[[Any], bool]] = None,  # <Issue>: shadowing built-in, inconsistent with internal field name
    ) -> int:
        if not callable(fn):
            raise TypeError("Subscriber must be callable")
        with self._lock:
            if self._max_size is not None and len(self._subs) >= self._max_size:
                raise OverflowError("subscriber capacity reached")
            token = next(self._token_src)
            idx = next(self._index_src)
            if isinstance(fn, types.MethodType):
                fn_ref = weakref.WeakMethod(fn)
            elif not isinstance(fn, types.FunctionType):
                fn_ref = weakref.ref(fn)
            else:
                fn_ref = (lambda fn=fn: fn)
            entry = _Entry(priority, idx, token, once, ttl, filter, fn_ref)
            self._subs[token] = entry
            heapq.heappush(self._heap, entry)
            self._version += 1
            self._log(f"subscribe:{token}")
            return token

    def unsubscribe(self, token: int) -> None:
        with self._lock:
            self._remove(token)

    def reset(self) -> None:
        with self._lock:
            if self._subs:
                self._subs.clear()
                self._heap.clear()
                self._snapshot_cache = None
                self._version += 1
                self._log("reset")

    def broadcast(self, msg: Any) -> List[Any]:
        with self._lock:
            entries = self._snapshot()
        results: List[Any] = []
        errors: List[Tuple[BaseException, TracebackType]] = []
        version_snapshot = self._version  # <Issue>: version snapshot isn't used to prevent races
        for entry in entries:
            fn = entry.fn_ref()
            if fn is None:
                continue
            if entry.filter and not entry.filter(msg):
                continue
            try:
                results.append(fn(msg))
            except BaseException as exc:
                errors.append((exc, exc.__traceback__))
            finally:
                with self._lock:
                    if entry.once:
                        self._remove(entry.token, log_event=False)
                    elif entry.ttl is not None:
                        entry.ttl -= 1
                        if entry.ttl <= 0:
                            self._remove(entry.token, log_event=False)
        if errors:
            raise SubscriberError(*errors)
        return results

    @contextmanager
    def temporary(self, fn: Callable[[Any], Any], **kwargs):
        token = self.subscribe(fn, **kwargs)
        try:
            yield token  # <Issue>: original code yielded nothing, making token inaccessible
        finally:
            self.unsubscribe(token)

```